# Comparing Regression Methods

In this notebook, I will compare the various regression methods (linear regression and penalized models, i.e. ridge and lasso) to predict hipcenter of the seatpos dataset with all other features as predictors. 

In [20]:
options(warn=-1)
library(faraway)
library(boot)
library(glmnet)
data(seatpos)
attach(seatpos)

The following objects are masked from seatpos (pos = 3):

    Age, Arm, hipcenter, Ht, HtShoes, Leg, Seated, Thigh, Weight

The following objects are masked from seatpos (pos = 4):

    Age, Arm, hipcenter, Ht, HtShoes, Leg, Seated, Thigh, Weight

The following objects are masked from seatpos (pos = 5):

    Age, Arm, hipcenter, Ht, HtShoes, Leg, Seated, Thigh, Weight

The following objects are masked from seatpos (pos = 10):

    Age, Arm, hipcenter, Ht, HtShoes, Leg, Seated, Thigh, Weight



In [40]:
# Linear regression
lm.model <- lm(hipcenter~.,data=seatpos)
summary(lm.model)


Call:
lm(formula = hipcenter ~ ., data = seatpos)

Residuals:
    Min      1Q  Median      3Q     Max 
-73.827 -22.833  -3.678  25.017  62.337 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)  
(Intercept) 436.43213  166.57162   2.620   0.0138 *
Age           0.77572    0.57033   1.360   0.1843  
Weight        0.02631    0.33097   0.080   0.9372  
HtShoes      -2.69241    9.75304  -0.276   0.7845  
Ht            0.60134   10.12987   0.059   0.9531  
Seated        0.53375    3.76189   0.142   0.8882  
Arm          -1.32807    3.90020  -0.341   0.7359  
Thigh        -1.14312    2.66002  -0.430   0.6706  
Leg          -6.43905    4.71386  -1.366   0.1824  
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 37.72 on 29 degrees of freedom
Multiple R-squared:  0.6866,	Adjusted R-squared:  0.6001 
F-statistic:  7.94 on 8 and 29 DF,  p-value: 1.306e-05


[1] 32.95203

In [41]:
# RMSE
sqrt(mean(lm.model$residuals^2))

[1] 32.95203

About 68.66% of the total variation is explained by the linear regression model. The RMSE is 32.95. None of the variables are statistically significant except the intercept. When predictor variables in the same regression model are correlated, they cannot independently predict the value of the dependent variable. I will assess the collinearity by looking at the correlation matrix, VIF and index condition number.

In [22]:
# Correlations
round(cor(seatpos[,-9]),2)

,Age,Weight,HtShoes,Ht,Seated,Arm,Thigh,Leg
Age,1.00,0.08,-0.08,-0.09,-0.17,0.36,0.09,-0.04
Weight,0.08,1.00,0.83,0.83,0.78,0.70,0.57,0.78
HtShoes,-0.08,0.83,1.00,1.00,0.93,0.75,0.72,0.91
Ht,-0.09,0.83,1.00,1.00,0.93,0.75,0.73,0.91
Seated,-0.17,0.78,0.93,0.93,1.00,0.63,0.61,0.81
Arm,0.36,0.70,0.75,0.75,0.63,1.00,0.67,0.75
Thigh,0.09,0.57,0.72,0.73,0.61,0.67,1.00,0.65
Leg,-0.04,0.78,0.91,0.91,0.81,0.75,0.65,1.00


In [23]:
# VIF
X=model.matrix(lm.model)[,-1]
vif(X)

Age     Weight    HtShoes         Ht     Seated        Arm      Thigh 
  1.997931   3.647030 307.429378 333.137832   8.951054   4.496368   2.762886 
       Leg 
  6.694291

In [24]:
# Index condition number
e = eigen(t(X)%*%X)
sqrt(e$val[1]/e$val)

[1]   1.00000  13.04226  20.10032 110.55123 156.91171 212.15650 261.66698
[8] 707.54911

There seems to be some evidences of multicollinearity in the model. From the correlation matrix, we can see that the correlation between the variables HtShoes and Ht is 1. These variables also seem to have a very high variance inflation factor (way greater than 10). The condition indices are also greater than 100, which indicates high multicollinearity. 

We will now update our model by stepwise dropping variables from our model.

In [25]:
step(lm.model)

Start:  AIC=283.62
hipcenter ~ Age + Weight + HtShoes + Ht + Seated + Arm + Thigh + 
    Leg

          Df Sum of Sq   RSS    AIC
- Ht       1      5.01 41267 281.63
- Weight   1      8.99 41271 281.63
- Seated   1     28.64 41290 281.65
- HtShoes  1    108.43 41370 281.72
- Arm      1    164.97 41427 281.78
- Thigh    1    262.76 41525 281.87
<none>                 41262 283.62
- Age      1   2632.12 43894 283.97
- Leg      1   2654.85 43917 283.99

Step:  AIC=281.63
hipcenter ~ Age + Weight + HtShoes + Seated + Arm + Thigh + Leg

          Df Sum of Sq   RSS    AIC
- Weight   1     11.10 41278 279.64
- Seated   1     30.52 41297 279.66
- Arm      1    160.50 41427 279.78
- Thigh    1    269.08 41536 279.88
- HtShoes  1    971.84 42239 280.51
<none>                 41267 281.63
- Leg      1   2664.65 43931 282.01
- Age      1   2808.52 44075 282.13

Step:  AIC=279.64
hipcenter ~ Age + HtShoes + Seated + Arm + Thigh + Leg

          Df Sum of Sq   RSS    AIC
- Seated   1     35.10 4131


Call:
lm(formula = hipcenter ~ Age + HtShoes + Leg, data = seatpos)

Coefficients:
(Intercept)          Age      HtShoes          Leg  
   456.2137       0.5998      -2.3023      -6.8297  


The model with the lowest AIC predicts hipcenter with age, htshoes, and leg as predictors. I will run a linear regression with these predictors.

In [26]:
lm.model1 <- lm(hipcenter ~ Age+HtShoes+Leg)
summary(lm.model1)


Call:
lm(formula = hipcenter ~ Age + HtShoes + Leg)

Residuals:
    Min      1Q  Median      3Q     Max 
-79.269 -22.770  -4.342  21.853  60.907 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept) 456.2137   102.8078   4.438 9.09e-05 ***
Age           0.5998     0.3779   1.587   0.1217    
HtShoes      -2.3023     1.2452  -1.849   0.0732 .  
Leg          -6.8297     4.0693  -1.678   0.1024    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 35.13 on 34 degrees of freedom
Multiple R-squared:  0.6813,	Adjusted R-squared:  0.6531 
F-statistic: 24.22 on 3 and 34 DF,  p-value: 1.437e-08


In [42]:
# RMSE
sqrt(mean(lm.model1$residuals^2))

[1] 33.22879

The updated model explains 68.13% of the total variation. With the reduced model, the variable HtShoes is now significant at the 10% level. Since HtShoes and Ht were collinear, I will re-run the regression by replacing the variable with Ht to demonstrate that both variables yield the same result.

In [27]:
lm.model2 <- lm(hipcenter ~ Age+Ht+Leg)
summary(lm.model2)


Call:
lm(formula = hipcenter ~ Age + Ht + Leg)

Residuals:
    Min      1Q  Median      3Q     Max 
-79.715 -22.758  -4.102  21.394  60.576 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept) 452.1976   100.9482   4.480 8.04e-05 ***
Age           0.5807     0.3790   1.532   0.1347    
Ht           -2.3254     1.2545  -1.854   0.0725 .  
Leg          -6.7390     4.1050  -1.642   0.1099    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 35.12 on 34 degrees of freedom
Multiple R-squared:  0.6814,	Adjusted R-squared:  0.6533 
F-statistic: 24.24 on 3 and 34 DF,  p-value: 1.426e-08


The results for both lm.model1 and lm.model2 are comparable.

Next I will run the penalized regression models to the dataset.

In [15]:
# Split data into training and test sets
train = 1:19
test = -train
training_data = seatpos[train,]
testing_data = seatpos[test,]

In [43]:
# Ridge regression
grid = 10^seq(10, -2, length = 100)
x <- model.matrix(hipcenter ~ ., data = training_data)
y <- model.matrix(hipcenter ~ ., data = testing_data)
z <- training_data$hipcenter
cv.out1 = cv.glmnet(x,z,alpha=0)
bestlam1 = cv.out1$lambda.min
ridge.model <- glmnet(x, z, alpha = 0, lambda=0.1)
ridge.pred <- predict(ridge.model, s = bestlam1, newx = y)

In [37]:
eval_results <- function(true, predicted, df) {
  SSE <- sum((predicted - true)^2)
  SST <- sum((true - mean(true))^2)
  R_square <- 1 - SSE / SST
  RMSE = sqrt(SSE/nrow(df))

data.frame(
  RMSE = RMSE,
  Rsquare = R_square
)}

eval_results(testing_data$hipcenter, ridge.pred, testing_data)

RMSE,Rsquare
48.21248,0.4432344


In [38]:
# Lasso regression
x <- model.matrix(hipcenter ~ ., data = training_data)
y <- model.matrix(hipcenter ~ ., data = testing_data)
z <- training_data$hipcenter
cv.out2 = cv.glmnet(x,z,alpha=0)
bestlam2 = cv.out2$lambda.min
lasso.model <- glmnet(x, z, alpha = 1,lambda=0.1)
lasso.pred <- predict(lasso.model, s = bestlam2, newx = y)

In [39]:
eval_results(testing_data$hipcenter, lasso.pred, testing_data)

RMSE,Rsquare
47.95326,0.4492053


The RMSE results for both ridge and lasso regressions are greater than OLS. The R-squared values are also lesser than that of OLS, indicating that the models explain less overall variability.